# ROS Multiclass

### Import pyspark

In [ ]:
from pyspark import SparkContext
sc=SparkContext('local[3]','KDD')

In [ ]:
import numpy as np

## Load data

Format of the data 


&nbsp; 
class column1 column2 column3 column4 ...... column_n 


&nbsp;
class column1 column2 column3 column4 ...... column_n 


&nbsp;
..       ..         ..        ..         ..          ..          .. 
 

&nbsp;
..        ..          ..          ..         ..         ..        .. 


&nbsp;

In [3]:
data=sc.textFile('/home/manuel/Escritorio/ROS y SMOTE SPARK Multiclase/Dataset_KDD23clases.txt')

In [4]:
data.take(2)

[u'4.0 1:0.13841313651560333 2:4.107214733400692E-4 3:0.002053607366700346 4:0.0016428858933602768 5:0.09734098918159641 6:0.6325110689437066 12:4.107214733400692E-4 15:4.107214733400692E-4 17:4.107214733400692E-4 18:4.107214733400692E-4 19:4.107214733400692E-4 23:4.107214733400692E-4 24:4.107214733400692E-4 29:4.107214733400692E-4 32:0.10473397570171765 33:0.01930390924698325 34:7.392986520121245E-5 35:8.214429466801383E-6 38:9.035872413481523E-5 39:1.3143087146882213E-4',
 u'4.0 1:0.18987261389181706 2:6.350254645211273E-4 3:0.0031751273226056365 4:0.002540101858084509 5:0.07112285202636626 6:0.5378665684493948 23:6.350254645211273E-4 24:6.350254645211273E-4 29:6.350254645211273E-4 32:0.16193149345288746 33:0.03048122229701411 34:1.2065483825901419E-4 35:1.2700509290422547E-5 38:1.39705602194648E-4 39:1.9685789400154946E-4']

In [5]:
data.count()

324102

In [6]:
#Separate the classes
data2=data.map(lambda s:s.split(' ')[0])#[0] Position of the class

In [7]:
#Count class matches
class_repetition=data2.flatMap(lambda s: s.split(" ")).map(lambda w: (w, 1)).reduceByKey(lambda a, b: a + b).sortBy(lambda s:s[1],False).collect()

In [8]:
xclass=[]
xtotals=[]

## Samples of each class

In [9]:
for x in range(len(class_repetition)):
    a=class_repetition[x]
    xclass.append(str(a[0]))
    xtotals.append(a[1])
    print a[0],a[1]
np_class=np.array(xclass)
np_total=np.array(xtotals)
    

14.0 278351
2.0 15892
20.0 12481
11.0 10413
13.0 2316
22.0 2203
17.0 1020
12.0 979
19.0 264
3.0 53
15.0 30
5.0 21
10.0 20
1.0 12
16.0 10
7.0 9
6.0 8
0.0 7
9.0 4
8.0 3
cuatro 2
cinco 2
4.0 2


In [10]:
rdd=sc.parallelize([])
aux=sc.parallelize([])

In [11]:
for i in range(len(class_repetition)):
    #Look for the class
    clas=data.filter(lambda s:s.split(' ')[0]==np_class[i])
    #Missing samples
    missing_samples=np_total[0]-np_total[i]
    #Block size to generate the random samples
    blocks=50000
    #Number of iterations per class
    iteration=missing_samples/blocks
    print("Class:",np_class[i],"Totals:",np_total[i],"Missing",missing_samples,"Iterations",iteration)
    if((iteration>0) and (i >0)):
        for j in range(iteration):
            #print(j+1)
            #Extract the sample
            extract=clas.takeSample(True,blocks,1)
            #Convert to rdd
            aux=sc.parallelize(extract)
            #join the samples of each class
            rdd=rdd.union(aux)
    #Join the classes
    rdd=rdd.union(clas)

('Class:', '14.0', 'Totals:', 278351, 'Missing', 0, 'Iterations', 0)
('Class:', '2.0', 'Totals:', 15892, 'Missing', 262459, 'Iterations', 5)
('Class:', '20.0', 'Totals:', 12481, 'Missing', 265870, 'Iterations', 5)
('Class:', '11.0', 'Totals:', 10413, 'Missing', 267938, 'Iterations', 5)
('Class:', '13.0', 'Totals:', 2316, 'Missing', 276035, 'Iterations', 5)
('Class:', '22.0', 'Totals:', 2203, 'Missing', 276148, 'Iterations', 5)
('Class:', '17.0', 'Totals:', 1020, 'Missing', 277331, 'Iterations', 5)
('Class:', '12.0', 'Totals:', 979, 'Missing', 277372, 'Iterations', 5)
('Class:', '19.0', 'Totals:', 264, 'Missing', 278087, 'Iterations', 5)
('Class:', '3.0', 'Totals:', 53, 'Missing', 278298, 'Iterations', 5)
('Class:', '15.0', 'Totals:', 30, 'Missing', 278321, 'Iterations', 5)
('Class:', '5.0', 'Totals:', 21, 'Missing', 278330, 'Iterations', 5)
('Class:', '10.0', 'Totals:', 20, 'Missing', 278331, 'Iterations', 5)
('Class:', '1.0', 'Totals:', 12, 'Missing', 278339, 'Iterations', 5)
('Class:

In [12]:
rdd.count()

5824102

In [13]:
data.count()

324102

## Save the new data

In [ ]:
rdd.coalesce(1).saveAsTextFile('/home/manuel/Escritorio/ros.dat')